## Setup 

In [1]:
%%capture
!pip install sktime
!pip install giotto-tda
!pip install netCDF4

In [2]:
%%capture
!wget http://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/sst.mon.mean.trefadj.anom.1880to2018.nc
!wget http://portal.nersc.gov/project/dasrepo/AGU_ML_Tutorial/nino34.long.anom.data.txt

In [10]:
%matplotlib inline
from typing import Tuple
import xarray as xr
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.express as px
import sklearn

## Cargando datos

In [11]:
# X => Cobe Sea-Surface Temperature Dataset:: this is a dataset of historical sea surface temperatures form 1880 to 2018
# y => Nino3.4 Indices: The Nino3.4 index measures the 3-month rolling average of equatorial Pacific Ocean temperature anomalies.

# The assemble_basic_predictors reads data as (time, lat, lon) and converts it to (time, lat*lon).
# This data would be X.
# Then y = load_enso_indices(), which are the actual values from the enso dataset.
# Note that the y columns is shifted by lead_time.

def load_enso_indices() -> pd.Series:
  """
  Reads in the txt data file to output a pandas Series of ENSO vals
  outputs
  -------
    pd.Series : monthly ENSO values starting from 1870-01-01
  """
  with open('nino34.long.anom.data.txt') as f:
    line = f.readline()
    enso_vals = []
    while line:
        yearly_enso_vals = map(float, line.split()[1:])
        enso_vals.extend(yearly_enso_vals)
        line = f.readline()

  enso_vals = pd.Series(enso_vals)
  enso_vals.index = pd.date_range('1870-01-01',freq='MS',
                                  periods=len(enso_vals))
  enso_vals.index = pd.to_datetime(enso_vals.index)
  return enso_vals


def assemble_basic_predictors_predictands(
    start_date,
    end_date,
    lead_time,
    use_pca=False,
    n_components=32
) -> Tuple[np.array]:
    """
    inputs
    ------

        start_date        str : the start date from which to extract sst
        end_date          str : the end date 
        lead_time         str : the number of months between each sst
                                value and the target Nino3.4 Index
        use_pca          bool : whether or not to apply principal components
                                analysis to the sst field
        n_components      int : the number of components to use for PCA

    outputs
    -------
        Returns a tuple of the predictors (np array of sst temperature anomalies) 
        and the predictands (np array the ENSO index at the specified lead time).

    """
    ds = xr.open_dataset('sst.mon.mean.trefadj.anom.1880to2018.nc')

    sst = ds['sst'].sel(time=slice(start_date, end_date))
    # 1.
    end_date = sst.time[-1].values
    num_time_steps = sst.shape[0]

    #sst is a 3D array: (time_steps, lat, lon)
    #in this tutorial, we will not be using ML models that take
    #advantage of the spatial nature of global temperature
    #therefore, we reshape sst into a 2D array: (time_steps, lat*lon)
    #(At each time step, there are lat*lon predictors)
    sst_values = sst.values.reshape(num_time_steps, -1)
    sst_values[np.isnan(sst_values)] = 0

    # PCA
    if use_pca:
        #GMZ: Turns the lat*lon dimension (i.e. 64800 dimension) into n_components
        pca = sklearn.decomposition.PCA(n_components=n_components)
        pca.fit(sst_values)
        X = pca.transform(sst_values)
    else:
        X = sst_values

    X = pd.DataFrame(X)
    X.index = sst.time

    start_date_plus_lead = pd.to_datetime(start_date) + \
                        pd.DateOffset(months=lead_time)

    end_date_plus_lead = pd.to_datetime(end_date) + \
                        pd.DateOffset(months=lead_time)

    y = load_enso_indices()
    y = y[y != -99.99]

    y = y[slice(start_date_plus_lead, end_date_plus_lead)]

    ds.close()
    return X, y

# 1. El problema es que "y" tiene datos hasta 2023 y X hasta 2018. Hence, si pasamos un end_date que sea mayor a los datos de X,
    # "y" tendría mas valores que X.
    # Para solucionar esto, la end_date para "y" será la ultima fecha que pudimos rescatar en X cuando le hicimos slice con el end_date.


In [12]:
load_enso_indices()

1870-01-01    -1.00
1870-02-01    -1.20
1870-03-01    -0.83
1870-04-01    -0.81
1870-05-01    -1.27
              ...  
2019-08-01   -99.99
2019-09-01   -99.99
2019-10-01   -99.99
2019-11-01   -99.99
2019-12-01   -99.99
Freq: MS, Length: 1800, dtype: float64

In [13]:
X, y = assemble_basic_predictors_predictands('1982-01-01','2023-01-01', 0, use_pca=True, n_components=1)

## TDA

In [14]:
from gtda.time_series import SingleTakensEmbedding
from gtda.plotting import plot_point_cloud
from gtda.homology import VietorisRipsPersistence
from sklearn.decomposition import PCA
from nolitsa import dimension, delay
import kmapper as km

### Encaje Takens

In [15]:
from typing import Tuple, List, Any

def get_takens_embedding(
    embedder: SingleTakensEmbedding,
    y: np.ndarray,
    pca: bool=False,
    verbose: bool=True
)-> Any:
    """Aplicar SingleTakensEmbeddings, donde el output es
    "y embedded" & y embedded" con dimensiones reducidas"""

    y_embedded = embedder.fit_transform(y)

    if verbose:
        print(f"Shape of embedded time series: {y_embedded.shape}")
        print(f"Optimal embedding dimension: {embedder.dimension_}")
        print(f"Optimal time delay: {embedder.time_delay_}")
        print(f"Stride: {embedder.stride}")

    if pca:
        pca = PCA(n_components=3)
        y_embedded_pca = pca.fit_transform(y_embedded)
        return y_embedded, y_embedded_pca

    return y_embedded


def get_persistence(y_embedded: np.array, dimensions: List[int], plot: bool = True):
    if y_embedded.ndim == 2:
        y_embedded = y_embedded[None, :, :]

    persistence = VietorisRipsPersistence(homology_dimensions=dimensions)
    if plot:
        return persistence.fit_transform_plot(y_embedded)
    else:
        return persistence.fit_transform(y_embedded)

In [16]:
embedder = SingleTakensEmbedding(
    parameters_type="search",
)

y_embedded, y_embedded_pca = get_takens_embedding(embedder, y, pca=True)

Shape of embedded time series: (441, 4)
Optimal embedding dimension: 4
Optimal time delay: 1
Stride: 1


### Nube de puntos

In [17]:
plot_point_cloud(y_embedded_pca)

In [18]:
dimensions = [0, 1]
persistence = get_persistence(y_embedded, dimensions)